Maggy enables you to train with Tensorflow distributed optimizers.
Using Maggy, you have to make minimal changes in train your model in a distributed fashion.

### 0. Spark Session

Make sure you have a running Spark Session/Context available.
On Hopsworks, just run your notebook to start the spark application.

In [1]:
from maggy import experiment

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
55,application_1619177783210_0001,pyspark,idle,Link,Link


SparkSession available as 'spark'.


### 1. Model definition

Let's define the model we want to train. The layers of the model have to be defined in the \_\_init__ function.

Do not instantiate the class, otherwise you won't be able to use Maggy.

In [2]:
# Model definition.
import tensorflow as tf
from tensorflow import keras
    
class NeuralNetwork(tf.keras.Model):

    def __init__(self, nlayers):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(28, 2, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(32, activation='relu')
        self.d2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = NeuralNetwork

### 2. Dataset creation

You can create the dataset here and pass it to the TfDistributedConfig, or creating it in the training function.

In this example, we are downloading the dataset from tensorflow.

In [3]:
import numpy as np

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.reshape(x_train[:20000,:,:], (10000, 28, 28, 1))
y_train = y_train[:10000]
x_test = np.reshape(x_test[:500,:,:], (500, 28, 28, 1))
y_test = y_test[:500]

11493376/11490434 [==============================] - 0s 0us/step

### 3. Defining the training function

The programming model is that you wrap the code containing the model training inside a wrapper function. Inside that wrapper function provide all imports and parts that make up your experiment.

The function should return the metric that you want to optimize for. This should coincide with the metric being reported in the Keras callback (see next point).
You can return the metric list, in this case only the loss element will be printed.

In [4]:
def training_function(model, train_set, test_set, hparams):

    from tensorflow import keras
    
    # Define training parameters
    num_epochs = 10
    batch_size = 256
    learning_rate = 0.1

    criterion = keras.losses.SparseCategoricalCrossentropy()
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate,momentum=0.9,decay=1e-5)
    
    model = model(nlayers = 2)
    
    model.compile(optimizer=optimizer, loss=criterion, metrics=["accuracy"])
    
    model.fit(train_set,
              batch_size=batch_size,
              epochs=num_epochs,
              )

    print("Testing")
    
    loss = model.evaluate(
        test_set,
        batch_size=32)
    
    return loss

### 4. Configuring the experiment

In order to use maggy distributed training, we have to configure the training model, we can pass it to TfDistributedConfig.
the model class has to be an implementation of __tf.keras.Model__.
We can also define __train_set__, __test_set__ and eventually the __model_parameters__. __model_parameters__ is a dictionary
containing the parameters to be used in the \_\_init__ function of your model.

In [5]:
from maggy.experiment_config.tf_distributed import TfDistributedConfig

#define the constructor parameters of your model
model_parameters = {
    'train_batch_size':
    5000,
    'test_batch_size': 250,
    
    'nlayers':2
}

#pass the model parameters in the last 
config = TfDistributedConfig(name="tf_test", 
                             module=model, 
                             train_set=(x_train, y_train), 
                             test_set=(x_test, y_test),
                             hparams=model_parameters
                            )

#### 5. Run distributed training

Finally, we are ready to launch the maggy experiment. You just need to pass 2 parameters: the training function and the configuration variable we defined in the previous steps.

In [6]:
experiment.lagom(training_function, config)

1: All executors registered: True
1: Epoch 1/10
1: 
1: 
0: All executors registered: True
0: Epoch 1/10
0: 
0: 
0: Epoch 2/10
1: Epoch 2/10
0: Epoch 3/10
1: Epoch 3/10
0: Epoch 4/10
1: Epoch 4/10
0: Epoch 5/10
1: Epoch 5/10
0: Epoch 6/10
1: Epoch 6/10
0: Epoch 7/10
1: Epoch 7/10
0: Epoch 8/10
1: Epoch 8/10
0: Epoch 9/10
1: Epoch 9/10
0: Epoch 10/10
1: Epoch 10/10
0: Testing
0: 
0: 
1: Testing
1: 
1: 
You are running Maggy on Hopsworks.
Final average test loss: 0.672
Finished experiment. Total run time: 0 hours, 0 minutes, 53 seconds
{'test result': 0.6719840168952942}
